# Day 16

In [1]:
from heapq import heappop, heappush

# Define directions
DIRECTIONS = ['E', 'S', 'W', 'N']  # Clockwise order: East, South, West, North
DELTA = {
    'E': (0, 1),  # East: move right
    'S': (1, 0),  # South: move down
    'W': (0, -1), # West: move left
    'N': (-1, 0)  # North: move up
}

def parse_maze(maze):
    start = end = None
    grid = []
    for r, line in enumerate(maze.strip().split('\n')):
        grid.append(line)
        if 'S' in line:
            start = (r, line.index('S'))
        if 'E' in line:
            end = (r, line.index('E'))
    return grid, start, end

def heuristic(pos, end):
    """Manhattan distance heuristic."""
    return abs(pos[0] - end[0]) + abs(pos[1] - end[1])

def reindeer_maze(maze):
    grid, start, end = parse_maze(maze)
    rows, cols = len(grid), len(grid[0])
    
    # Priority queue for A* (score, position, direction)
    pq = []
    heappush(pq, (0, start, 'E'))  # Start facing East with score 0
    
    # Visited states (position, direction) -> lowest score
    visited = {}
    visited[(start, 'E')] = 0

    while pq:
        score, (x, y), facing = heappop(pq)

        # If we reach the end, return the score
        if (x, y) == end:
            return score

        # Try moving forward
        dx, dy = DELTA[facing]
        nx, ny = x + dx, y + dy
        if 0 <= nx < rows and 0 <= ny < cols and grid[nx][ny] != '#':
            new_score = score + 1
            if (nx, ny, facing) not in visited or new_score < visited[(nx, ny, facing)]:
                visited[(nx, ny, facing)] = new_score
                heappush(pq, (new_score, (nx, ny), facing))

        # Try rotating left or right
        for rotation in [-1, 1]:  # -1 for left, 1 for right
            new_facing = DIRECTIONS[(DIRECTIONS.index(facing) + rotation) % 4]
            new_score = score + 1000
            if (x, y, new_facing) not in visited or new_score < visited[(x, y, new_facing)]:
                visited[(x, y, new_facing)] = new_score
                heappush(pq, (new_score, (x, y), new_facing))

# Example usage
maze1 = """
###############
#.......#....E#
#.#.###.#.###.#
#.....#.#...#.#
#.###.#####.#.#
#.#.#.......#.#
#.#.#####.###.#
#...........#.#
###.#.#####.#.#
#...#.....#.#.#
#.#.#.###.#.#.#
#.....#...#.#.#
#.###.#.#.#.#.#
#S..#.....#...#
###############
"""
maze2 = """
#################
#...#...#...#..E#
#.#.#.#.#.#.#.#.#
#.#.#.#...#...#.#
#.#.#.#.###.#.#.#
#...#.#.#.....#.#
#.#.#.#.#.#####.#
#.#...#.#.#.....#
#.#.#####.#.###.#
#.#.#.......#...#
#.#.###.#####.###
#.#.#...#.....#.#
#.#.#.#####.###.#
#.#.#.........#.#
#.#.#.#########.#
#S#.............#
#################
"""

print(reindeer_maze(maze1))  # Output: 7036
print(reindeer_maze(maze2))  # Output: 11048

with open("Input/InputDay16P1.txt", "r") as f:
    Inputdata = f.read()

print(reindeer_maze(Inputdata))


7036
11048
75416


In [3]:
from collections import deque
from heapq import heappop, heappush

DIRECTIONS = ['E', 'S', 'W', 'N']  # Clockwise: East, South, West, North
DELTA = {
    'E': (0, 1),  # East: move right
    'S': (1, 0),  # South: move down
    'W': (0, -1), # West: move left
    'N': (-1, 0)  # North: move up
}

def parse_maze(maze):
    start = end = None
    grid = []
    for r, line in enumerate(maze.strip().split('\n')):
        grid.append(line)
        if 'S' in line:
            start = (r, line.index('S'))
        if 'E' in line:
            end = (r, line.index('E'))
    return grid, start, end

def heuristic(pos, end):
    """Manhattan distance heuristic."""
    return abs(pos[0] - end[0]) + abs(pos[1] - end[1])

def find_best_score(maze):
    """Find the best score from S to E using A*."""
    grid, start, end = parse_maze(maze)
    rows, cols = len(grid), len(grid[0])
    
    # Priority queue for A* (score, position, direction)
    pq = []
    heappush(pq, (0, start, 'E'))  # Start facing East with score 0
    
    # Visited states (position, direction) -> lowest score
    visited = {}
    visited[(start, 'E')] = 0

    while pq:
        score, (x, y), facing = heappop(pq)

        # If we reach the end, return the score
        if (x, y) == end:
            return score

        # Try moving forward
        dx, dy = DELTA[facing]
        nx, ny = x + dx, y + dy
        if 0 <= nx < rows and 0 <= ny < cols and grid[nx][ny] != '#':
            new_score = score + 1
            if (nx, ny, facing) not in visited or new_score < visited[(nx, ny, facing)]:
                visited[(nx, ny, facing)] = new_score
                heappush(pq, (new_score, (nx, ny), facing))

        # Try rotating left or right
        for rotation in [-1, 1]:  # -1 for left, 1 for right
            new_facing = DIRECTIONS[(DIRECTIONS.index(facing) + rotation) % 4]
            new_score = score + 1000
            if (x, y, new_facing) not in visited or new_score < visited[(x, y, new_facing)]:
                visited[(x, y, new_facing)] = new_score
                heappush(pq, (new_score, (x, y), new_facing))

def find_best_path_tiles(maze):
    """Find all tiles that are part of any best path."""
    grid, start, end = parse_maze(maze)
    rows, cols = len(grid), len(grid[0])
    
    # Get the best score
    best_score = find_best_score(maze)
    
    # Perform BFS to backtrack from end to start
    queue = deque([(end, 0)])  # (position, score)
    visited = set()
    visited.add(end)
    
    best_path_tiles = set()
    best_path_tiles.add(end)

    while queue:
        (x, y), score = queue.popleft()

        # Explore all neighbors (reverse of the forward movement and rotation logic)
        for direction, (dx, dy) in DELTA.items():
            nx, ny = x - dx, y - dy  # Reverse movement
            if 0 <= nx < rows and 0 <= ny < cols and grid[nx][ny] != '#' and (nx, ny) not in visited:
                new_score = score + 1
                if new_score <= best_score:  # Only explore valid paths
                    queue.append(((nx, ny), new_score))
                    visited.add((nx, ny))
                    best_path_tiles.add((nx, ny))

    return len(best_path_tiles)

# Example usage
maze1 = """
###############
#.......#....E#
#.#.###.#.###.#
#.....#.#...#.#
#.###.#####.#.#
#.#.#.......#.#
#.#.#####.###.#
#...........#.#
###.#.#####.#.#
#...#.....#.#.#
#.#.#.###.#.#.#
#.....#...#.#.#
#.###.#.#.#.#.#
#S..#.....#...#
###############
"""
maze2 = """
#################
#...#...#...#..E#
#.#.#.#.#.#.#.#.#
#.#.#.#...#...#.#
#.#.#.#.###.#.#.#
#...#.#.#.....#.#
#.#.#.#.#.#####.#
#.#...#.#.#.....#
#.#.#####.#.###.#
#.#.#.......#...#
#.#.###.#####.###
#.#.#...#.....#.#
#.#.#.#####.###.#
#.#.#.........#.#
#.#.#.#########.#
#S#.............#
#################
"""

print(find_best_path_tiles(maze1))  # Should match problem's example: 45
print(find_best_path_tiles(maze2))  # Should match problem's example: 64


104
132
